In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Initialise Model
m = gp.Model("7-4-2")


# Initialise Decision Variables using a list of indices from 1 to 420
indices = list(range(0, 420))

beta_size = 10  # B0, ... ,B9
indices_2 = list(range(0, beta_size))

x = m.addVars(indices, vtype=GRB.BINARY, name='x')
c = m.addVars(indices, lb = 0, vtype=GRB.CONTINUOUS, name='c') 
z = m.addVars(indices, vtype=GRB.INTEGER, name='z')
y = m.addVars(indices, lb = 0, vtype=GRB.CONTINUOUS, name='y')
beta = m.addVars(indices_2, lb = 0, vtype=GRB.INTEGER, name='Beta')


# Set Objective function
m.setObjective(y.sum())


# Define the alpha matrix
import itertools
# Create all unique pairs for the face-up cards (left and right)
card_values = range(1, 8)  # Card values from 1 to 7
face_up_pairs = list(itertools.permutations(card_values, 2))

# Generate alpha matrix
alpha_matrix = []
condensed_matrix = []


for left, right in face_up_pairs:
    for hidden1, hidden2 in itertools.combinations(card_values, 2):
        if hidden1 not in (left, right) and hidden2 not in (left, right): 
            alpha_i1 = left
            alpha_i2 = right  
            alpha_i3 = 1 if left > right else 0
            alpha_i4 = 1 if hidden1 > left else 0
            alpha_i5 = 1 if hidden1 > right else 0
            alpha_j6 = hidden1
            alpha_i7 = 1 if hidden2 > left else 0
            alpha_i8 = 1 if hidden2 > right else 0
            alpha_j9 = hidden2
            
            alpha_matrix.append([alpha_i1, alpha_i2, alpha_i3, alpha_i4, alpha_i5, alpha_j6, alpha_i7, alpha_i8, alpha_j9])   # j6 and j9 are the values of hidden 1 and hidden 2 respectively

            sorted_row = sorted([alpha_i1, alpha_i2, alpha_j6, alpha_j9])
            condensed_matrix.append(sorted_row)

# alpha_matrix
# condensed_matrix

# This is to for ease of coding out the matching constraints (see the variable 'new_matrix')
from collections import defaultdict

# Initialize a dictionary to store the rows grouped by their elements
grouped_rows = defaultdict(list)

# Iterate through the condensed matrix
for index, row in enumerate(condensed_matrix):
    # Convert the row to a tuple to make it hashable and usable as a dictionary key
    key = tuple(row)
    # Append the row index to the list corresponding to the key
    grouped_rows[key].append(index)

# Define the size of the new matrix
num_rows = len(grouped_rows)
num_cols = 12  # Assuming all rows have the same number of columns

# Initialize a new matrix with zeros
new_matrix = [[0] * num_cols for _ in range(num_rows)]

# Iterate through the grouped rows
for row_index, indices in enumerate(grouped_rows.values()):
    # Fill the new matrix with indices
    for col_index, index in enumerate(indices):
        new_matrix[row_index][col_index] = index

# Print the new matrix
for row in new_matrix:
    print(row)


# Create constraints
P = 13 

for i in range(420):
    m.addConstr(y[i] + (1 - x[i]) * P >= c[i])
    m.addConstr(c[i] >=  (1 - x[i]))
    m.addConstr(c[i] <=  P - 1)    
    
for i in range(35):
    m.addConstr(gp.quicksum(x[new_matrix[i][j]] for j in range(12)) == 1)

for i in range(42):
    m.addConstr(gp.quicksum(x[10*i + j] for j in range(10)) <= 1, 'x_sum_{}'.format(i))

for i in range(420):
    m.addConstr(c[i] == beta[0] + gp.quicksum(beta[k+1] * alpha_matrix[i][k] for k in range(0, 9)) - z[i] * P)


# Additional Cuts
m.addConstr(beta.sum() >= 1)

for i in range(10):
    m.addConstr(beta[i] <= P-1)  # P = 13 for (7-4-2) model

m.addConstr(beta[6] >= 1)
m.addConstr(beta[9] >= 1)


# Solve the Model
m.setParam('TimeLimit', 10*60) # Set a time limit of 30 minutes
m.optimize()



[0, 10, 20, 60, 70, 80, 120, 130, 140, 180, 190, 200]
[1, 11, 30, 61, 71, 90, 121, 131, 150, 240, 250, 260]
[2, 12, 40, 62, 72, 100, 122, 132, 160, 300, 310, 320]
[3, 13, 50, 63, 73, 110, 123, 133, 170, 360, 370, 380]
[4, 21, 31, 64, 81, 91, 181, 191, 210, 241, 251, 270]
[5, 22, 41, 65, 82, 101, 182, 192, 220, 301, 311, 330]
[6, 23, 51, 66, 83, 111, 183, 193, 230, 361, 371, 390]
[7, 32, 42, 67, 92, 102, 242, 252, 280, 302, 312, 340]
[8, 33, 52, 68, 93, 112, 243, 253, 290, 362, 372, 400]
[9, 43, 53, 69, 103, 113, 303, 313, 350, 363, 373, 410]
[14, 24, 34, 124, 141, 151, 184, 201, 211, 244, 261, 271]
[15, 25, 44, 125, 142, 161, 185, 202, 221, 304, 321, 331]
[16, 26, 54, 126, 143, 171, 186, 203, 231, 364, 381, 391]
[17, 35, 45, 127, 152, 162, 245, 262, 281, 305, 322, 341]
[18, 36, 55, 128, 153, 172, 246, 263, 291, 365, 382, 401]
[19, 46, 56, 129, 163, 173, 306, 323, 351, 366, 383, 411]
[27, 37, 47, 187, 212, 222, 247, 272, 282, 307, 332, 342]
[28, 38, 57, 188, 213, 232, 248, 273, 292, 367

In [4]:
for i in range(10):
    print('beta[{}]: {}'.format(i, beta[i].X))

# Print the values of the 'c' variables
for i in range(420):
    print('c[{}]: {}'.format(i, c[i].X))



# Initialize a counter for zero
zero_count = 0
one_count = 0

for i in range(420):
     if c[i].X == 0.0:
        zero_count += 1
     if c[i].X == 1.0:
        one_count += 1
         
print(zero_count)
print(one_count)


# Print the values of the 'x' variables
zero_count = 0

for i in range(420):
    # print('x[{}]: {}'.format(i, x[i].X))
    if (x[i].X == 1):
        print('x[{}]: {}'.format(i, x[i].X), 'c[{}]: {}'.format(i, c[i].X), alpha_matrix[i])
        if c[i].X == 0.0:
            zero_count += 1

print(zero_count)

beta[0]: 0.0
beta[1]: 0.0
beta[2]: 0.0
beta[3]: 2.0
beta[4]: 9.0
beta[5]: 2.0
beta[6]: 2.0
beta[7]: 2.0
beta[8]: 9.0
beta[9]: 2.0
c[0]: 10.0
c[1]: 12.0
c[2]: 1.0
c[3]: 3.0
c[4]: 1.0
c[5]: 3.0
c[6]: 5.0
c[7]: 5.0
c[8]: 7.0
c[9]: 9.0
c[10]: 6.0
c[11]: 8.0
c[12]: 10.0
c[13]: 12.0
c[14]: 1.0
c[15]: 3.0
c[16]: 5.0
c[17]: 5.0
c[18]: 7.0
c[19]: 9.0
c[20]: 8.0
c[21]: 8.0
c[22]: 10.0
c[23]: 12.0
c[24]: 10.0
c[25]: 12.0
c[26]: 1.0
c[27]: 5.0
c[28]: 7.0
c[29]: 9.0
c[30]: 8.0
c[31]: 10.0
c[32]: 10.0
c[33]: 12.0
c[34]: 12.0
c[35]: 12.0
c[36]: 1.0
c[37]: 1.0
c[38]: 3.0
c[39]: 9.0
c[40]: 8.0
c[41]: 10.0
c[42]: 12.0
c[43]: 12.0
c[44]: 12.0
c[45]: 1.0
c[46]: 1.0
c[47]: 3.0
c[48]: 3.0
c[49]: 5.0
c[50]: 8.0
c[51]: 10.0
c[52]: 12.0
c[53]: 1.0
c[54]: 12.0
c[55]: 1.0
c[56]: 3.0
c[57]: 3.0
c[58]: 5.0
c[59]: 7.0
c[60]: 12.0
c[61]: 1.0
c[62]: 3.0
c[63]: 5.0
c[64]: 3.0
c[65]: 5.0
c[66]: 7.0
c[67]: 7.0
c[68]: 9.0
c[69]: 11.0
c[70]: 8.0
c[71]: 10.0
c[72]: 12.0
c[73]: 1.0
c[74]: 1.0
c[75]: 3.0
c[76]: 5.0
c[77]: 5.